# Atom-Atom Mapping

Given two molecular structures, it is important to be able to identify atoms that are chemically similar. This a commonly used in 3D QSAR pharmacore analysis, substructure searching, metabolic pathway identification, and chemical machine learning.

The code block below shows how easily the *Procrustes* library can be used to map atoms of *but-1-en-3-yne* (**A**) and *3,3-dimethylpent-1-en-4-yne* (**B**) as depicted in **Fig. (i)**. Based on our chemical intuition, we can tell that the triple and double bonds of both molecules "match" one another; however, simple (geometric) molecular alignment based on three-dimensional coordinates does not identify that. The pivotal step is defining a representation that contains bonding information, and then using permutation Procrustes to match atoms between the two chemical structures. 

Inspired by graph theory, we represented each molecule with an "adjacency" matrix where the diagonal elements are the atomic numbers and the off-diagonal elements are the bond orders (matrix $\mathbf{A} \in \mathbb{R}^{4 \times 4}$ and $\mathbf{B} \in \mathbb{R}^{7 \times 7}$ in **Fig. (ii)** ). The two-sided permutation Procrustes (`Procrustes.permutation_2sided`) with one-transformation can be used to find the optimal matching of the two matrices.

![Fig. 1. Atom-atom Mapping with Two-sided Permutation Procrustes](notebook_data/atom_atom_mapping/atom_atom_mapping.png "Fig. 1 Atom-atom Mapping with Two-sided Permutation Procrustes")

It is important to note that the permutation Procrustes requires the two matrices to be of the 
same size, so the smaller matrix $\mathbf{A}$ is padded with zero rows and columns to have same 
shape as matrix $\mathbf{B}$. After obtaining the optimal permutation matrix $\mathbf{P}$, the 
transformed matrix $\mathbf{P^{\top}AP}$ should be compared to matrix $\mathbf{B}$ for 
identifying the matching atoms; the zero rows/columns correspond to atoms in $\mathbf{B}$ for 
which there are no corresponding atoms in $\mathbf{A}$. The mapping between atoms can be also 
directly deduced from matrix $\mathbf{P}$,


\begin{equation}
\min_{\mathbf{P}} {\left\lVert \mathbf{P}^{\top} \mathbf{A} 
\mathbf{P} - \mathbf{B} \right\rVert}_{F}^2
\end{equation}

In [1]:
# import required libraries
import numpy as np

from procrustes import permutation_2sided

Now we have two matrices $A$ and $B$ for two molecules,

In [2]:
# Define molecule A, but‐1‐en‐3‐yne
A = np.array([[6, 3, 0, 0],
              [3, 6, 1, 0],
              [0, 1, 6, 2],
              [0, 0, 2, 6]])

In [3]:
# Define molecule B, 3,3‐dimethylpent‐1‐en‐4‐yne
B = np.array([[6, 3, 0, 0, 0, 0, 0],
              [3, 6, 1, 0, 0, 0, 0],
              [0, 1, 6, 1, 0, 1, 1],
              [0, 0, 1, 6, 2, 0, 0],
              [0, 0, 0, 2, 6, 0, 0],
              [0, 0, 1, 0, 0, 6, 0],
              [0, 0, 1, 0, 0, 0, 6]])

Now we have both matrix $A$ and $B$ defined and we can use two-sided permutation procrustes to find the optimal transformation operation.

In [4]:
# two-sided permutation Procrustes
result = permutation_2sided(A, B, 
                            method="approx-normal1",
                            single=True, pad=True)

In [5]:
# the permutation matrix
P = result.t
print(P)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]


The new matrix of molecule A can be obtained with the permutation operation.

In [6]:
# Compute the transformed molecule A
new_A = np.dot(P.T, np.dot(result.new_a, P)).astype(int)
# compare to B
print("Transformed A: \n", new_A)

Transformed A: 
 [[6 3 0 0 0 0 0]
 [3 6 0 1 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 6 2 0 0]
 [0 0 0 2 6 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


The computed result is shown in the **Fig. (iii)***, generating ideal matching of the double and triple carbon-carbon bonds. The new matrix representation of $\mathbf{A}$ suggests that atom 3 is empty since the third row and third column of $\mathbf{A}$ are zero (matrix elements in blue). That is, a virtual atom 3 was added to molecule $\mathbf{A}$ to align with atom 3 in molecule $\mathbf{B}$. Similarly, atoms 6 and 7 in molecule $\mathbf{B}$ (matrix elements in blue) do not have meaningful matches in $\mathbf{A}$, and are mapped to two virtual atoms, atom 6 and 7 in molecule $\mathbf{A}$. 